In [4]:
import numpy as np 

x = np.random.rand(10, 1, 28, 28)
x.shape

(10, 1, 28, 28)

In [6]:
x[0].shape

(1, 28, 28)

In [7]:
x[1].shape

(1, 28, 28)

In [8]:
x[0, 0]

array([[0.36219248, 0.22197811, 0.34433574, 0.9865006 , 0.87876886,
        0.0699013 , 0.06080222, 0.29899482, 0.48004087, 0.02977771,
        0.62343239, 0.21699433, 0.45771038, 0.03725692, 0.90103636,
        0.30424666, 0.09478219, 0.02688751, 0.64507554, 0.95130156,
        0.70440037, 0.31070964, 0.67505492, 0.84579266, 0.07077881,
        0.34875943, 0.89753879, 0.92350047],
       [0.26040641, 0.6864873 , 0.86505323, 0.5080419 , 0.50246864,
        0.02230003, 0.8731537 , 0.70054272, 0.10325551, 0.99174476,
        0.65380435, 0.72837636, 0.06345839, 0.75993036, 0.29595811,
        0.30342928, 0.0994953 , 0.03936153, 0.20919747, 0.16166553,
        0.10930699, 0.04689067, 0.45292636, 0.22500049, 0.1422863 ,
        0.13489995, 0.07362477, 0.31216686],
       [0.647324  , 0.15956331, 0.5324695 , 0.1797269 , 0.58682526,
        0.62415807, 0.22447639, 0.84509605, 0.68692629, 0.19921324,
        0.54901453, 0.87187655, 0.12270746, 0.94992805, 0.01563639,
        0.59822275, 0.2127

In [10]:
import sys, os
sys.path.append(os.pardir)
from common.util import im2col

x1 = np.random.rand(1, 3, 7, 7) # (데이터 수, 채널 수, 높이, 너비)
col1 = im2col(x1, 5, 5, stride=1, pad=0)
print(col1.shape) # (9, 75)


x2 = np.random.rand(10, 3, 7, 7) # (데이터 수, 채널 수, 높이, 너비)
col2 = im2col(x2, 5, 5, stride=1, pad=0)
print(col2.shape) # (90, 75)

(9, 75)
(90, 75)


In [ ]:
class Convolution:
    def __init__(self, W, b, stride=1, pad=0):
        self.W = W
        self.b = b
        self.stride = stride
        self.pad = pad

    def forward(self, x):
        FN, C, FH, FW = self.W.shape
        N, C, H, W = x.shape
        out_h = 1 + int((H + 2*self.pad - FH) / self.stride)
        out_w = 1 + int((W + 2*self.pad - FW) / self.stride)

        col = im2col(x, FH, FW, self.stride, self.pad)
        col_W = self.W.reshape(FN, -1).T

        out = np.dot(col, col_W) + self.b
        out = out.reshape(N, out_h, out_w, -1).transpose(0, 3, 1, 2)

        return out

In [ ]:
class Pooling:
    def __init__(self, pool_h, pool_w, stride=2, pad=0):
        self.pool_h = pool_h
        self.pool_w = pool_w
        self.stride = stride
        self.pad = pad
        
    def forward(self, x):
        N, C, H, W = x.shape
        out_h = int(1 + (H - self.pool_h) / self.stride)
        out_w = int(1 + (W - self.pool_w) / self.stride)

        col = im2col(x, self.pool_h, self.pool_w, self.stride, self.pad)
        col = col.reshape(-1, self.pool_h*self.pool_w)

        out = np.max(col, axis=1)
        out = out.reshape(N, out_h, out_w, C).transpose(0, 3, 1, 2)

        return out

In [ ]:
# coding: utf-8
import sys, os
sys.path.append(os.pardir)  # 親ディレクトリのファイルをインポートするための設定
import numpy as np
import matplotlib.pyplot as plt
from dataset.mnist import load_mnist
from simple_convnet import SimpleConvNet
from common.trainer import Trainer

# データの読み込み
(x_train, t_train), (x_test, t_test) = load_mnist(flatten=False)

# 処理に時間のかかる場合はデータを削減 
#x_train, t_train = x_train[:5000], t_train[:5000]
#x_test, t_test = x_test[:1000], t_test[:1000]

max_epochs = 20

network = SimpleConvNet(input_dim=(1,28,28), 
                        conv_param = {'filter_num': 30, 'filter_size': 5, 'pad': 0, 'stride': 1},
                        hidden_size=100, output_size=10, weight_init_std=0.01)
                        
trainer = Trainer(network, x_train, t_train, x_test, t_test,
                  epochs=max_epochs, mini_batch_size=100,
                  optimizer='Adam', optimizer_param={'lr': 0.001},
                  evaluate_sample_num_per_epoch=1000)
trainer.train()

# パラメータの保存
network.save_params("params.pkl")
print("Saved Network Parameters!")

# グラフの描画
markers = {'train': 'o', 'test': 's'}
x = np.arange(max_epochs)
plt.plot(x, trainer.train_acc_list, marker='o', label='train', markevery=2)
plt.plot(x, trainer.test_acc_list, marker='s', label='test', markevery=2)
plt.xlabel("epochs")
plt.ylabel("accuracy")
plt.ylim(0, 1.0)
plt.legend(loc='lower right')
plt.show()

train loss:2.2993584810170056
=== epoch:1, train acc:0.211, test acc:0.263 ===
train loss:2.2973067255802047
train loss:2.2928070373676244
train loss:2.2882520750082342
train loss:2.279968771003474
train loss:2.266013548764573
train loss:2.2483173210295537
train loss:2.229114968261498
train loss:2.2144311769476035
train loss:2.1989865813146294
train loss:2.132839440020116
train loss:2.120046903539519
train loss:2.066370409983871
train loss:2.0408318761896367
train loss:1.9931523743232982
train loss:1.9346978516569369
train loss:1.8612973204748195
train loss:1.7325785414061585
train loss:1.7223909654424334
train loss:1.6726682615192687
train loss:1.4893449167526795
train loss:1.3960683429679266
train loss:1.3714023858405349
train loss:1.3650269540826334
train loss:1.200281811184321
train loss:1.1684039572173988
train loss:1.064112210861803
train loss:0.9187067821337354
train loss:0.9409098550636772
train loss:0.9582821845634577
train loss:1.1342741119389848
train loss:0.8233274108076201

train loss:0.20633421434309976
train loss:0.2166588690100918
train loss:0.1915460417905063
train loss:0.2138653001819042
train loss:0.10291892750537965
train loss:0.14938791587645392
train loss:0.2645839826765279
train loss:0.17211836334130623
train loss:0.2478808602364666
train loss:0.12510164632254822
train loss:0.20376621998456282
train loss:0.22320804260915172
train loss:0.17559250318044228
train loss:0.2797641784059763
train loss:0.25976718722849934
train loss:0.1279841079919756
train loss:0.20361032663684475
train loss:0.13957827497194494
train loss:0.17146084403993014
train loss:0.1814203479242275
train loss:0.2173224245789308
train loss:0.2750911450682449
train loss:0.18445744228555963
train loss:0.12452624552485726
train loss:0.18849349658057968
train loss:0.2650850294608735
train loss:0.1518380509159147
train loss:0.22298406680212196
train loss:0.19227323946163236
train loss:0.2538832987180509
train loss:0.19073594525140294
train loss:0.1491918982678996
train loss:0.177436234

train loss:0.17062722995947607
train loss:0.14584616176242451
train loss:0.1318182656481328
train loss:0.04234458934240151
train loss:0.11166776745177172
train loss:0.14090361469199592
train loss:0.16398683553606097
train loss:0.17463861345653625
train loss:0.14352165596302893
train loss:0.20511527399002397
train loss:0.13671043250078935
train loss:0.16429202607133192
train loss:0.16168519860361702
train loss:0.12526449495668518
train loss:0.1101432939152049
train loss:0.10849909908989525
train loss:0.13408655818413714
train loss:0.098080855317695
train loss:0.1270937565324247
train loss:0.09021154730360395
train loss:0.04298243777623733
train loss:0.10993912356987323
train loss:0.1543480793124154
train loss:0.12786773652701494
train loss:0.1368757225416576
train loss:0.07335827445344477
train loss:0.06562843132361477
train loss:0.1128356431095364
train loss:0.13415620857234864
train loss:0.20504063086843233
train loss:0.07342415376247467
train loss:0.1517519004303143
train loss:0.1887

train loss:0.17282822693974623
train loss:0.15338160694321612
train loss:0.0789205713811634
train loss:0.09504327898959136
train loss:0.04360479770059588
train loss:0.11027523609437187
train loss:0.09632759407776842
train loss:0.06899187298193217
train loss:0.06692977021854724
train loss:0.1012580359347195
train loss:0.10517000652949808
train loss:0.09329386120060644
train loss:0.06557052273562503
train loss:0.06147967911096571
train loss:0.09541984732222937
train loss:0.05178878230451033
train loss:0.08757249754700065
train loss:0.05113547840413063
train loss:0.04252235640660669
train loss:0.10167655128138214
train loss:0.18831040435052476
train loss:0.04895901180286951
train loss:0.0635693675191728
train loss:0.12428939638127127
train loss:0.10379318732171815
train loss:0.05929154271904066
train loss:0.18622437840060804
train loss:0.07755027041860005
train loss:0.03338056425099022
train loss:0.08150944874983178
train loss:0.10568903261193402
train loss:0.049685618738501296
train loss

train loss:0.02707803618634043
train loss:0.0529094123462433
train loss:0.04159638813013055
train loss:0.05830184897162542
train loss:0.10042199041666834
train loss:0.032405136903095426
train loss:0.12751186377747165
train loss:0.03076883399071677
train loss:0.06243533535547201
train loss:0.07198696641496741
train loss:0.07910926761395355
train loss:0.038025699013331346
train loss:0.179828050952538
train loss:0.07599941043927973
train loss:0.008449309388560117
train loss:0.08473529028880977
train loss:0.11440390429861104
train loss:0.06765866238850134
train loss:0.08860649951093723
train loss:0.139362170445871
train loss:0.1382792176968649
train loss:0.048622476216030025
train loss:0.05870428415781435
train loss:0.10194278111615855
train loss:0.08456430284075532
train loss:0.07032096687634358
train loss:0.061644068927686604
train loss:0.11932763225530424
train loss:0.09279093025273774
train loss:0.031242805679499507
train loss:0.06738425094205165
train loss:0.03323290663673484
train lo

...

train loss:0.003359330147539198
train loss:0.009061556872098014
train loss:0.04675192292536416
train loss:0.01654130486524142
train loss:0.012512594227235815
train loss:0.0019917350229171297
train loss:0.0022856177791852985
train loss:0.0011541173933366016
train loss:0.008153722201188105
train loss:0.0014527844588938285
train loss:0.0021144997373141023
train loss:0.007145204953506005
train loss:0.002271215933155708
train loss:0.002421104211970862
train loss:0.004554108027681952
train loss:0.0013832378543450715
train loss:0.005399939524654895
train loss:0.004802848557525619
train loss:0.010507792926640041
train loss:0.0015697633148203205
train loss:0.00019981877254461068
train loss:0.0010461910449843983
train loss:0.0010205663485213434
train loss:0.0010103670041552652
train loss:0.01052037996423186
train loss:0.003637293617209432
=== epoch:11, train acc:0.997, test acc:0.986 ===
train loss:0.013549790634977023
train loss:0.002237648004654655
train loss:0.00922795689641846
train loss:0.0

train loss:0.0043097479849033195
train loss:0.0011735677516173403
train loss:0.0011554029068803593
train loss:0.0025989384754234214
train loss:0.00785797562580502
train loss:0.0031830255068606463
train loss:0.0008812860573784909
train loss:0.004690339378129732
train loss:0.005601519263342467
train loss:0.002769183050602314
train loss:0.0038851257473664995
train loss:0.007678636861009176
train loss:0.0031379280092280375
train loss:0.002647853686548367
train loss:0.0032355211819959035
train loss:0.0008677153885072279
train loss:0.0039618609773466726
train loss:0.0014984312000036895
train loss:0.00033431468321175074
train loss:0.0014458244072990084
train loss:0.07972177203024057
train loss:0.0016006128615354844
train loss:0.012898213952082755
train loss:0.0013952887296863072
train loss:0.001861520572398871
train loss:0.00511023730120775
train loss:0.002244156183804704
train loss:0.011614160091674914
train loss:0.00432788936429528
train loss:0.004729523968520815
train loss:0.00683218417518

train loss:0.0012938000621985188
train loss:0.0017285042808580467
train loss:0.0025573962552830347
train loss:0.0051813868414457035
train loss:0.004218455434007101
train loss:0.005742870877511643
train loss:0.0032360961069490984
train loss:0.010074203014096417
train loss:0.005202259866902264
train loss:0.004482109026559742
train loss:0.0024135091288194585
train loss:0.005020399905497133
train loss:0.00833699322644607
train loss:0.0015709983760208268
train loss:0.0003283829539428545
train loss:0.0024385825715117337
train loss:0.0018366588683178884
train loss:0.008489354524369525
train loss:0.0013997327539334399
train loss:0.003292783653093023
train loss:0.010177449442254927
train loss:0.0012141503734937446
train loss:0.010546423779333236
train loss:0.0008129385045822909
train loss:0.013431591601547414
train loss:0.0026539665569773846
train loss:0.004017851599317693
train loss:0.0019419107563397237
train loss:0.0026219788440736756
train loss:0.0025028293761671584
train loss:0.00731151900

train loss:0.0011435739533268631
train loss:0.016607902083442695
train loss:0.004231863623467241
train loss:0.0031164139186934336
train loss:0.0010923111819114088
train loss:0.004091900184699083
train loss:0.0026037505630498244
train loss:0.011849361108308096
train loss:0.004034395975576239
train loss:0.06902652281605295
train loss:0.0396844575633382
train loss:0.0010984060441765397
train loss:0.005231574945626094
train loss:0.005128123015748104
train loss:0.0038427418279899156
train loss:0.0037401577551330195
train loss:0.008214993566676243
train loss:0.0059654199209372705
train loss:0.006296930848905023
train loss:0.017877916016813024
train loss:0.00443341158556574
train loss:0.007188963807803853
train loss:0.010542406695682722
train loss:0.004404665473796869
train loss:0.0005319845796611445
train loss:0.005839585200694421
train loss:0.005687776187771769
train loss:0.002698626637037155
train loss:0.0468713859808996
train loss:0.00562893865702082
train loss:0.021137109354640252
train 

train loss:0.002988523792228111
train loss:0.001049497521046095
train loss:0.00754258407739603
train loss:0.002979730687937388
train loss:0.03468569049292521
train loss:0.009032253853110843
train loss:0.0018019264087521354
train loss:0.003018099235123929
train loss:0.006432465858333705
train loss:0.01036303514576082
train loss:0.0071844196741522185
train loss:0.005641923384982973
train loss:0.002439799717948544
train loss:0.007363019528666902
train loss:0.0013960070703990463
train loss:0.006329038742216636
train loss:0.0011467285855053626
train loss:0.0015100937222162162
train loss:0.004381493704704964
train loss:0.0008914073200221555
train loss:0.008865385541529444
train loss:0.003878597152603497
train loss:0.0005914605270572607
train loss:0.0041639176388079315
train loss:0.004106185793692139
train loss:0.0063910422154424594
train loss:0.0023187837986302228
train loss:0.030783764431317237
train loss:0.0035992262148509474
train loss:0.011649785195571702
train loss:0.0034101134574309527